## Artificial and Computational Intelligence Assignment 2

## Gaming with Min-Max Algorithm - Solution template

### List only the BITS (Name) of active contributors in this assignment:
<ol>
    <li>ARUN MATHEW  <b>(2021sc04982)</b>
<li>KANURU NAGESWARA RAO <b> (2021sc04876)</b>
<li>THAKKAR PRACHI CHETAN CHETNA <b>(2021sc04875)</b>
</ol>

### Coding begins here

#### Define constants :

In [1]:
#Code Block
import random
import copy
from numpy import inf
import sys

# Define constants
WHITE = 'white'
BLACK = 'black'
HUMAN = 'human'
COMP = 'computer'

# The cost associated with each piece. Rook has highest value followed by knight and pawn
EVAL_PARAMS = {
    'ROOK':50,
    'KNIGHT':30,
    'PAWN':10
}

# Allowed direction for movement of Pawn. This variable is used to prevent Pawn from moving backwards
COLOR_DIR = {
    WHITE:1,
    BLACK:-1
}

# The images used to represent the pieces in the ASCII chess board
images = {
    WHITE : {"Pawn" : "♙", "Rook" : "♖", "Knight" : "♘"}, 
    BLACK : {"Pawn" : "♟", "Rook" : "♜", "Knight" : "♞"}
}

# Define the allowed movements, diagonal, cardinal and L-movements are the only allowed movements in a chess board
diagonal_movement = [(-1,-1),(-1,1),(1,-1),(1,1)]
cardinal_movement = [(-1,0),(0,-1),(0,1),(1,0)]
l_movement = [(-2,-1),(-2,1),(-1,-2),(-1,2),(1,-2),(1,2),(2,-1),(2,1)]


#### Utility Functions :

In [2]:
# Fuction to change the color. If black is provided as input, white is returned as output and vice versa
def switch_color(this_color):
    return WHITE if this_color == BLACK else BLACK

# A filter function used to filter dictionaries
def filter_pair(d, filter_string):
    for key, val in d.items(): 
        if val is None:
            continue
        if not val.color == filter_string:
            continue
    yield key, val

### Implementation of the Min-Max algorithm

In [3]:
# Minimax algorithm used by the AI for deciding the best movement of the pieces. 
# This uses the board.evaluate function to determine the board state
def minimax(board, depth, maximizing_player, maximizing_color):
    
    best_move = None
    
    if depth == 0 or board.gameover:
        return best_move, board.evaluate(maximizing_color)
    
    moves = board.get_moves(maximizing_color)
    if len(moves) > 0:
        best_move = random.choice(moves)
    
    if  maximizing_player:
        max_eval = -inf
        for move in moves:
            board_copy = copy.deepcopy(board)
            board.move(move['current_position'], move['next_move'])
            eval_mov, current_eval = minimax(board, depth -1, False, switch_color(maximizing_color))
            board.reset(board_copy)
            if current_eval > max_eval:
                max_eval = current_eval
                best_move = move
        return best_move, max_eval
    else:
        min_eval = inf
        for move in moves:
            board_copy = copy.deepcopy(board)
            board.move(move['current_position'], move['next_move'])
            eval_mov, current_eval = minimax(board, depth -1, True, maximizing_color)
            board.reset(board_copy)
            if current_eval < min_eval:
                min_eval = current_eval
                best_move = move
        return best_move, min_eval


### Creation of chess board of 3x3 size :

In [4]:
class Board():
    def __init__(self, size, turn=None):
        self.gameboard = {}
        self.gameover = False
        self.current_turn = turn
        for i in range(0,3):
            for j in range(0,3):
                self.gameboard[(i,j)] =None
    
    def reset(self, board):
        self.gameboard = board.gameboard
        self.current_turn = board.current_turn
    
    # Create an ASCII representation of the board
    def print_board(self):
        print('--------------------')        
        print("  1 | 2 | 3 | ")
        for i in range(0,3):
            print("-"*12)
            print(chr(i+97),end="|")
            for j in range(0,3):
                item = self.gameboard.get((i,j)," ")
                if item is None or item == '':
                    if i == 0:
                        pad = '.'
                    else:
                        pad = '-'
                    print(pad+' |', end = " ")
                else:
                    print(item.image+'|', end = " ")
            print()
        print("-"*12)
        print('--------------------') 
        
        
    def set_turn(self,turn):
        self.current_turn = turn

    def add_piece(self,piece, position):
        self.gameboard[position] = piece

    def move(self, original_position, new_position):
        piece = self.gameboard[original_position]
        self.gameboard[new_position] = piece
        self.gameboard[original_position] = None
    
    def gameStatus(self):
        white_coins = self.find_all_coins(WHITE)
        black_coins = self.find_all_coins(BLACK)
        
        if len(white_coins) == 0:
            return "WINNER: BLACK"
        elif len(black_coins) == 0:
            return "WINNER: WHITE"
        elif not self.is_playable():
            return "STALEMATE"
        else:
            return "CONTINUE"
            
    def is_playable(self):
        white_coins = self.find_all_coins(WHITE)
        black_coins = self.find_all_coins(BLACK)
        white_moves = []
        black_moves = []
        if len(white_coins)>0:
            white_moves = self.get_moves(WHITE)
        if len(black_coins)>0 == 0:
            black_moves = self.get_moves(BLACK)
        return len(white_moves)>0 and len(black_moves)>0
        
    def whiteScore(self):
        color_coins = self.find_all_coins(WHITE)
        total_score = 0
        for key, value in color_coins.items():
            total_score += value.score()
        return total_score
    
    def blackScore(self):
        color_coins = self.find_all_coins(BLACK)
        total_score = 0
        for key, value in color_coins.items():
            total_score += value.score()
        return total_score
    
# The board static evaluation function. 
# It evaluates the board based on the maximizing color
    def evaluate(self, maximizing_color):
        if maximizing_color == WHITE:
            return self.whiteScore() - self.blackScore()
        else:
            return self.blackScore() - self.whiteScore()

    def next_turn(self):
        self.current_turn = switch_color(self.current_turn)
        return self.current_turn
    
    def current_turn(self):
        self.current_turn
        
    def enemy_position(self, loc, color):
        coin = self.gameboard[loc]
        return coin is not None and coin.color != color
        
    def can_move(self,old_position,new_position, color, attack=False, st_cut_allowed=True):
        coin = self.gameboard[new_position]
        if coin is None:
            return True
        elif coin.color == color:
            return False
        elif not st_cut_allowed:
            return False
        else:
            return True
        
    
    def find_coin(self, coin_char, color):
        cname = None
        if coin_char.upper() == 'P':
            cname = 'Pawn'
        elif coin_char.upper() == 'K':
            cname = 'Knight'
        elif coin_char.upper() == 'R':
            cname = 'Rook'
        else:
            cname = 'Unknown'
        
        for pos,c in self.gameboard.items():
            if c is not None and c.__class__.__name__ == cname and c.color == color:
                return pos, c
            
        return (-1,-1), None

    
    def find_all_coins(self, color):
        new_dict = {}
        for key, value in self.gameboard.items():
            if value is not None:
                if color == value.color:
                    new_dict[key] = value
        return new_dict
    
    def get_moves(self,color):
        all_moves = []
        color_coins = self.find_all_coins(color)
        for key, value in color_coins.items():
            moves = value.availableMoves(key[0], key[1], self)
            for m in moves:
                d = {
                    'color':color,
                    'type':value,
                    'current_position':key,
                    'next_move':m
                }
                all_moves.append(d)
        return all_moves
        

### Create data structures for pieces :

#### Define parent class for Pieces

In [5]:
class Piece():
    def __init__(self, color, image):
        self.color = color
        self.image = image
    
    def get_color(self):
        return self.color
    
    def score(self):
        return 0
    
    def set_position(self, new_position):
        if (self.is_valid(new_position)):
            self.position = new_position

    def availableMoves(self,x,y,gameboard):
        print("ERROR: Movements are not defined for generic classes")

    def is_valid(self,x,y):
        return x >=0  and x<3 and y >=0  and y<3
    
    def next_location(self, position, movement):
        return tuple(ele1 + ele2 for ele1, ele2 in zip(position, movement))

#### Define Rook, Knight and Pawn (Individual Pieces)

In [6]:
class Knight(Piece):

    def __init__(self,color):
        img = images[color]["Knight"]
        super().__init__(color, img)
    
    def __str__(self):
        return 'Name='+self.__class__.__name__+' '+' Color='+self.color+' '+' Image='+self.image 
    
    def score(self):
        return 30
    
    def availableMoves(self,x,y,gameboard):  
        position = (x,y)
        answers = []
        
         # handle stoppage if another coin of same color is present
        for mov in l_movement:
            valid = True
            next_loc = position
            while valid:
                next_loc = super().next_location(next_loc, mov)
                log = str(next_loc)
                if super().is_valid(next_loc[0],next_loc[1]) and gameboard.can_move(position,next_loc,self.color, attack=False ): 
                    valid = True
                    log += ' is valid:true '
                    answers.append(next_loc)
                    if gameboard.enemy_position(next_loc, self.color):
                        log += ' enemy position:true '
                        continue
                        valid = False
                else:
                    log += ' is valid:false '
                    valid = False
        return answers

        
class Rook(Piece):
    def __init__(self,color):
        super().__init__(color,images[color]["Rook"])
        
    def __str__(self):
        return 'Name='+self.__class__.__name__+' '+' Color='+self.color+' '+' Image='+self.image 

    def score(self):
        return 50
    
    def availableMoves(self,x,y,gameboard):
        position = (x,y)
        answers = []
        
        # handle stoppage if another coin of same color is present
        for mov in cardinal_movement:
            valid = True
            next_loc = position
            while valid:
                next_loc = super().next_location(next_loc, mov)
                log = str(next_loc)
                if super().is_valid(next_loc[0],next_loc[1]) and gameboard.can_move(position,next_loc,self.color, attack=False ): 
                    valid = True
                    log += ' is valid:true '
                    answers.append(next_loc)
                    if gameboard.enemy_position(next_loc, self.color):
                        log += ' enemy position:true '
                        continue
                        valid = False
                else:
                    log += ' is valid:false '
                    valid = False            
        return answers
    

class Pawn(Piece):
    
    def __init__(self,color):
        img = images[color]["Pawn"]
        super().__init__(color, img)
    
    def __str__(self):
        return 'Name='+self.__class__.__name__+' '+' Color='+self.color +' '+' Image='+self.image 
    
    def score(self):
        return 10
    
    
    def availableMoves(self,x,y,gameboard):
        position = (x,y)
        answers = []
        
        movement = tuple(COLOR_DIR[self.color] * elem for elem in (1,0))
        mv = self.next_location(position,movement)
        if super().is_valid(mv[0], mv[1]) and gameboard.can_move(position,mv,self.color, attack=False, st_cut_allowed=False ):
            answers.append(mv)
        
        #attack movements
        att_moves = []
        att_moves.append( tuple(COLOR_DIR[self.color] * elem for elem in (1,-1)) )
        att_moves.append( tuple(COLOR_DIR[self.color] * elem for elem in (1,1)) )
        
        for move in att_moves:
            next_loc = super().next_location(position,move)
            if super().is_valid(next_loc[0],next_loc[1]) and gameboard.enemy_position(next_loc, self.color):
                answers.append(next_loc)

        return answers

### Create data structures for Game :

In [7]:
class Game:
    def __init__(self, operator_color):
        self.gameboard = Board(3)
        self.operator_color = color
        self.computer_color = switch_color(color)
        self.set_board()
        self.finished = False
        print('Computer has chosen:',self.computer_color) 
    
    def start(self, color):
        self.print_board()
        print('Starting the game with color:',color)
        self.gameboard.set_turn(color)
        try:
            self.run_game()
        except:
            print('Closing Game')
        
    def print_board(self):
        self.gameboard.print_board()
    
    def run_game(self):
        while not self.finished:
            if self.is_computer_turn():
                print('Computer''s turn. Color:',self.computer_color)
                self.make_next_move()
            else:
                print('Operator''s turn. Color:',self.operator_color)
                self.ask_and_move()
            self.gameboard.next_turn()

    def is_computer_turn(self):
        return self.gameboard.current_turn != self.operator_color
    
    def make_next_move(self):
        if not self.gameboard.is_playable():
            print('No more move allowed. Exiting game')
            self.finished = True
            self.gameboard.gameover = True
            print('Game status',self.gameboard.gameStatus() )
            return        
            
        move, eval_cost = minimax(self.gameboard, 3, True, self.computer_color)
        if move is not None:
            self.gameboard.move(move['current_position'], move['next_move'])
        
        self.print_board()
    
    def is_legal(self, coin, color, old_position, position):
        if coin is not None:
            moves = coin.availableMoves(old_position[0], old_position[1], self.gameboard)
            if position in moves:
                return True
            else:
                print('Invalid move')
                return False
    
    def ask_and_move(self):
        moved = False
        counter = 1
        if not self.gameboard.is_playable():
            print('No more move allowed. Exiting game')
            self.finished = True
            self.gameboard.gameover = True
            print('Game status',self.gameboard.gameStatus() )
            return
            
        while not moved and counter < 5:
            coin, new_position = self.ask_for_input()
            pos, c = self.gameboard.find_coin(coin, self.gameboard.current_turn)
            if self.is_legal(c, self.gameboard.current_turn, pos, new_position):
                self.gameboard.move(pos, new_position)
                self.gameboard.print_board()
                moved = True
                counter = counter + 1
    
    def ask_for_input(self):
        coin = input('Choose what coin you want to move (K [knight], R[Rook], P[Pawn] ). [Note: Use q to quit]')
        if coin.lower() == 'q':
            print('Exiting')
            self.finished = True
            self.gameboard.gameover = True
            sys.exit("Terminating game");
        
        x = -1
        y = -1
        cell_no = input('Chose the new position (Eg: A1, B3):')
        allowed_chars = ['a','b','c']
        allowed_nums = ['1','2','3']
        if len(cell_no) == 2:
            try:
                x = allowed_chars.index(cell_no[0].lower())
                y = allowed_nums.index(cell_no[1])
            except ValueError:
                x = -1
                y = -1
        return coin, (int(x),int(y))

    def set_board(self):
        #(0,0) - white pawn in black cell
        self.gameboard.add_piece(Pawn(WHITE), (0,0) )
        #(0,1) - white knight in black cell
        self.gameboard.add_piece(Knight(WHITE),(0,1) )
        #(0,2) - white Rook in black cell
        self.gameboard.add_piece(Rook(WHITE), (0,2))
    
        
        #(2,1) - white pawn in black cell
        self.gameboard.add_piece(Pawn(BLACK),(2,1))
        #(2,2) - white knight in black cell
        self.gameboard.add_piece(Knight(BLACK),(2,2))

#### Terminal State / Game Ending (Win/Loss/Draw):  Design 

The is_playable() function of gameboard determines whether game can be continued or not. If there are no playable positions or if there are no pieces left for a color, the game will stop and the game status is evaluated.


        if not self.gameboard.is_playable():
            print('No more move allowed. Exiting game')
            self.finished = True
            self.gameboard.gameover = True
            print('Game status',self.gameboard.gameStatus() )
            return      

### Terminal State / Game Ending (Win/Loss/Draw):  Implementation

The following implementation in the Board class determines whether further movement is possible


    def is_playable(self):
        white_coins = self.find_all_coins(WHITE)
        black_coins = self.find_all_coins(BLACK)
        white_moves = []
        black_moves = []
        if len(white_coins)>0:
            white_moves = self.get_moves(WHITE)
        if len(black_coins)>0 == 0:
            black_moves = self.get_moves(BLACK)
        return len(white_moves)>0 and len(black_moves)>0    

#### Dynamic Input Design : 
<p>
1. To start the game, the operator should choose a color in the input box. Ex: white or black.
   <br>Computer will automatically choose the other color.
</p>
<p>   
2. Decide who is starting the game. If the operator starts the game, give the chosen color of the operator.  Ex: white or black.
</p>
<p> 
3. The chess board is divided into rows and columns. 
    <br>The rows are 'a','b', and 'c'
    <br>The columns are '1','2', and '3'
</p>
<p> 
4. A cell can be represented using the alphabet and the number combination.
   <br>Ex: To select the cell with co-ordinates second row and first column, use b1
</p>
<p> 
5. When it is the operator's turn, give the movement instruction in the input boxes
    <br>Choose what coin you want to move (K [knight], R[Rook], P[Pawn] ): P
    <br>Choose the new position (Eg: A1, B3):b2
    <p>
    <i>
        <ul>
        <li>Note: when providing the coin selection, use abbrevations K, R or P to choose Knight, Rook or Pawn respectively
        <li>Note: You can enter 'q' to quit the game at this stage
        </ul>
    </i>
    </p>
<p> 
6. Computer makes the movement based on the operator's selection
</p>

### Proper Prompt to the User for Dynamic Input & Handling incorrect inputs - Implement

In [8]:
print('Starting the game1.')
color = input('Choose the operator color: ')
turn = input('Choose whose turn is this (white/black): ')

Starting the game1.
Choose the operator color: white
Choose whose turn is this (white/black): white


In [9]:
g = Game(color)
g.start(turn)

Computer has chosen: black
--------------------
  1 | 2 | 3 | 
------------
a|♙| ♘| ♖| 
------------
b|- | - | - | 
------------
c|- | ♟| ♞| 
------------
--------------------
Starting the game with color: white
Operators turn. Color: white
Choose what coin you want to move (K [knight], R[Rook], P[Pawn] ). [Note: Use q to quit]K
Chose the new position (Eg: A1, B3):C3
--------------------
  1 | 2 | 3 | 
------------
a|♙| . | ♖| 
------------
b|- | - | - | 
------------
c|- | ♟| ♘| 
------------
--------------------
Computers turn. Color: black
--------------------
  1 | 2 | 3 | 
------------
a|♙| . | ♖| 
------------
b|- | ♟| - | 
------------
c|- | - | ♘| 
------------
--------------------
Operators turn. Color: white
Choose what coin you want to move (K [knight], R[Rook], P[Pawn] ). [Note: Use q to quit]P
Chose the new position (Eg: A1, B3):B2
--------------------
  1 | 2 | 3 | 
------------
a|. | . | ♖| 
------------
b|- | ♙| - | 
------------
c|- | - | ♘| 
------------
-------------

### Any Inference / Documentation

#### Alpha Beta Pruning

Alpha-Beta Pruning was NOT used in the implementation due to the following reasons:
1. The depth of the tree was not very large and the system's processing time was very minimal. Adding Alpha-Beta pruning would have increased the code complexity without giving any added advantage.
2. There was no specific instructions to include Alpha Beta Pruning in the implemetation. 



#### The following websites and documentations were refered during development of the code
<ul>
    <li>https://medium.com/@SereneBiologist/the-anatomy-of-a-chess-ai-2087d0d565</li>
    <li>https://www.freecodecamp.org/news/simple-chess-ai-step-by-step-1d55a9266977/</li>
</ul>

### Conclusion

With the minimax algorithm, the computer is able to determine the best move for the board state. 